# Scratchpad for paper revisions

In [1]:
%load_ext autoreload
%autoreload 2
import pickle
import os, sys
root_path = os.path.realpath('../')
sys.path.append(root_path)

import torch
from pathlib import Path

from utils.data import make_dataset
from utils.nnet import get_device

from hebbcl.logger import MetricLogger
from hebbcl.model import Nnet
from hebbcl.trainer import Optimiser, train_model
from hebbcl.parameters import parser
from hebbcl.tuner import HPOTuner

In [2]:
# obtain params
args = parser.parse_args(args=[])

# set checkpoint directory
save_dir = (
        Path("checkpoints") / "test_allhebb"
    ) 

# get device (gpu/cpu)
args.device = get_device(args.cuda)[0]
dict(sorted(vars(args).items(),key=lambda k: k[0]))


{'centering': True,
 'ctx_avg': True,
 'ctx_avg_alpha': 1,
 'ctx_avg_type': 'ema',
 'ctx_avg_window': 50,
 'ctx_scaling': 2,
 'ctx_w_init': 0.5,
 'ctx_weights': False,
 'cuda': False,
 'device': device(type='cpu'),
 'gating': 'oja_ctx',
 'hebb_normaliser': 10.0,
 'hpo_fixedseed': False,
 'hpo_scheduler': None,
 'hpo_searcher': None,
 'log_interval': 50,
 'loss_funct': 'reward',
 'lrate_hebb': 0.01,
 'lrate_sgd': 0.01,
 'n_episodes': 200,
 'n_features': 27,
 'n_hidden': 100,
 'n_out': 1,
 'n_runs': 50,
 'perform_hebb': True,
 'perform_sgd': True,
 'save_dir': 'simu1',
 'save_results': True,
 'seed': 1234,
 'training_schedule': 'blocked',
 'verbose': True,
 'weight_init': 0.01}

## hyperparameter optimisation
hpo on network trained with fewer episodes

In [16]:
args = parser.parse_args(args=[])
args.n_episodes = 8
args.hpo_fixedseed = True
args.hpo_scheduler = "bohb"
args.hpo_searcher = "bohb"
dict(sorted(vars(args).items(),key=lambda k: k[0]))

{'centering': True,
 'ctx_avg': True,
 'ctx_avg_alpha': 1,
 'ctx_avg_type': 'ema',
 'ctx_avg_window': 50,
 'ctx_scaling': 2,
 'ctx_w_init': 0.5,
 'ctx_weights': False,
 'cuda': False,
 'gating': 'oja_ctx',
 'hebb_normaliser': 10.0,
 'hpo_fixedseed': True,
 'hpo_scheduler': 'bohb',
 'hpo_searcher': 'bohb',
 'log_interval': 50,
 'loss_funct': 'reward',
 'lrate_hebb': 0.01,
 'lrate_sgd': 0.01,
 'n_episodes': 8,
 'n_features': 27,
 'n_hidden': 100,
 'n_out': 1,
 'n_runs': 50,
 'perform_hebb': True,
 'perform_sgd': True,
 'save_dir': 'simu1',
 'save_results': True,
 'seed': 1234,
 'training_schedule': 'blocked',
 'verbose': True,
 'weight_init': 0.01}

In [20]:
# init tuner
tuner = HPOTuner(args, time_budget=60*15, metric="loss")

2022-06-23 13:34:42,250	INFO services.py:1374 -- View the Ray dashboard at http://127.0.0.1:8265
2022-06-23 13:34:42,839	INFO packaging.py:353 -- Creating a file package for local directory 'D:\DPHIL_02_CONTLEARN\paper\codebase\utils'.
2022-06-23 13:34:42,857	INFO packaging.py:222 -- Pushing file package 'gcs://_ray_pkg_f415459db55ddc24.zip' (0.12MiB) to Ray cluster...
2022-06-23 13:34:42,860	INFO packaging.py:225 -- Successfully pushed file package 'gcs://_ray_pkg_f415459db55ddc24.zip'.
2022-06-23 13:34:42,892	INFO packaging.py:353 -- Creating a file package for local directory 'D:\DPHIL_02_CONTLEARN\paper\codebase\hebbcl'.
2022-06-23 13:34:42,910	INFO packaging.py:222 -- Pushing file package 'gcs://_ray_pkg_99d9078df640c3e1.zip' (0.06MiB) to Ray cluster...
2022-06-23 13:34:42,913	INFO packaging.py:225 -- Successfully pushed file package 'gcs://_ray_pkg_99d9078df640c3e1.zip'.
2022-06-23 13:34:42,917	INFO packaging.py:353 -- Creating a file package for local directory '../ray_tune/'.
2

(pid=None) 2022-06-23 13:34:44,987	INFO working_dir.py:85 -- Setup working dir for gcs://_ray_pkg_f3330b0ae3f3fee2.zip


In [ ]:
# HPO on blocked with oja (all units)
tuner.tune(n_samples=500)

In [22]:
tuner.results[["config.lrate_sgd", "config.lrate_hebb","config.ctx_scaling","config.seed","mean_acc","mean_loss","done"]].head(15)

,config.lrate_sgd,config.lrate_hebb,config.ctx_scaling,config.seed,mean_acc,mean_loss,done
trial_id,,,,,,,
ddc1ed9d,0.002057,0.007351,2.0,7853.0,0.500,-0.004615,False
df05b844,0.000675,0.000657,6.0,8018.0,0.575,0.042206,False
e0335d0e,0.001184,0.042442,7.0,5009.0,0.525,-0.108420,False
e1ead1ec,0.001290,0.013744,5.0,5611.0,0.500,0.124506,False
e32cafcf,0.020820,0.000110,4.0,8826.0,0.500,0.050307,False
e5139d09,0.000168,0.007018,3.0,3688.0,0.500,-0.116148,False
e5538008,0.001555,0.008998,7.0,7887.0,0.500,0.053909,False
e555cf70,0.040493,0.005062,3.0,4361.0,0.500,-0.096960,False
e5583f25,0.012965,0.000270,6.0,7045.0,0.500,-0.041142,False


In [43]:
# tuner.results.sort_values("mean_acc",ascending=False).head(20)
df = tuner.results
df = df[["mean_loss", "mean_acc", "config.lrate_sgd","config.lrate_hebb", "config.ctx_scaling","config.seed","done"]]
df = df[df["done"]==True]
df = df.drop(columns=["done"])
df = df.dropna()
df = df.sort_values("mean_loss",ascending=True)

df.reset_index()
df.head(15)

,mean_loss,mean_acc,config.lrate_sgd,config.lrate_hebb,config.ctx_scaling,config.seed
trial_id,,,,,,
06078292,-27.878670,1.000,0.078666,0.013706,2.0,9797.0
ef3d4f55,-27.792665,1.000,0.083653,0.003193,3.0,3433.0
06464a96,-27.585148,0.975,0.088811,0.003293,3.0,9625.0
0a5cd5ee,-27.268438,1.000,0.088232,0.006905,3.0,9299.0
11fd386c,-27.167648,0.950,0.099984,0.043230,1.0,7787.0
06247ee1,-26.320683,0.975,0.083810,0.007017,3.0,7975.0
31a59b80,-22.772160,0.950,0.067418,0.006831,3.0,5945.0
1fe2c637,-17.945683,0.900,0.075429,0.025832,2.0,7530.0
115dfb56,-14.617438,0.750,0.066006,0.023437,2.0,8157.0


In [26]:
tuner.best_cfg

{'lrate_sgd': 0.07866581741161789,
 'lrate_hebb': 0.013705922079653616,
 'ctx_scaling': 2,
 'seed': 9797}

In [44]:
with open("../results/raytune_oja_ctx_blocked_8episodes.pkl", "wb") as f:
    pickle.dump(df, f)

### verify results

In [45]:
import numpy as np
import random
import torch
# obtain params
args = parser.parse_args(args=[])

# set checkpoint directory
save_dir = (
        Path("checkpoints") / "test_allhebb"
    ) 

# get device (gpu/cpu)
args.device = get_device(args.cuda)[0]

# override defaults 
args.n_episodes = 8
args.lrate_hebb = tuner.best_cfg["lrate_hebb"]
args.lrate_sgd = tuner.best_cfg["lrate_sgd"]
args.ctx_scaling = tuner.best_cfg["ctx_scaling"]

# np.random.seed(tuner.best_cfg["seed"])
# random.seed(tuner.best_cfg["seed"])
# torch.manual_seed(tuner.best_cfg["seed"])


# create dataset 
dataset = make_dataset(args)

# instantiate logger, model and optimiser:
logger = MetricLogger(save_dir)
model = Nnet(args)
optimiser = Optimiser(args)

# send model to device (GPU?)
model = model.to(args.device)


# train model
train_model(args, model, optimiser, dataset, logger)

step 0, loss: task a 0.0031, task b -0.0428 | acc: task a 0.5000, task b 0.5000
... n_a: 8 n_b: 5
step 50, loss: task a -8.2112, task b -0.1023 | acc: task a 1.0000, task b 0.5000
... n_a: 10 n_b: 0
step 100, loss: task a -12.9730, task b -0.0549 | acc: task a 1.0000, task b 0.5000
... n_a: 19 n_b: 0
step 150, loss: task a -14.1218, task b 0.0596 | acc: task a 1.0000, task b 0.5000
... n_a: 29 n_b: 2
step 200, loss: task a -14.4646, task b -0.1773 | acc: task a 1.0000, task b 0.6000
... n_a: 31 n_b: 1
step 250, loss: task a -14.2353, task b -4.3225 | acc: task a 1.0000, task b 0.8000
... n_a: 26 n_b: 3
step 300, loss: task a -14.2857, task b -11.8683 | acc: task a 1.0000, task b 1.0000
... n_a: 24 n_b: 11
step 350, loss: task a -14.3328, task b -13.9667 | acc: task a 1.0000, task b 1.0000
... n_a: 24 n_b: 18
done


In [46]:
print(f"config: lrate_sgd: {args.lrate_sgd:.4f}, lrate_hebb: {args.lrate_hebb:.4f}, context offset: {args.ctx_scaling}")
print(f"terminal accuracy: {logger.acc_total[-1]:.2f}, loss: {logger.losses_total[-1]:.2f}")

config: lrate_sgd: 0.0787, lrate_hebb: 0.0137, context offset: 2
terminal accuracy: 1.00, loss: -28.30


In [ ]:
df = tuner.results
